# Prepoznavanje oblika

Autor: Djordje Marjanovic 2016/0120

## Priprema

### Ucitavanje Biblioteka

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

## Zadatak 1

Za bazu rukom pisanih samoglasnika, koja je dostupna na sajtu predmeta*
isprojektovati inovativni sistem za prepoznavanje cifara zasnovan na
testiranju hipoteza. Rezultate prikazati u obliku matrice konfuzije. Izveštaj
treba da sadrži kratki opis isprojektovanog sistema, obrazložen izbor
obeležja, kao i karakteristične primere pravilno i nepravilno klasifikovanih
slova.